# Keypoint representation and correspondance

##### *3D Dental Similarity Quantification in Forensic Odontology Identification - repository* 

This notebook calculates keypoint representation and saves it as json files. Afterwards the landmark correspondance function can be used to evaluate keypoint correspondance

## Initialization

In [ ]:
#Import
import keypoint_detection as kpd   #The "no print" version
import os
import vedo
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from scipy.spatial import cKDTree
from sklearn.preprocessing import normalize
import pyshot
import time
import json

method_name = "temp"


In [ ]:
##### CALCULATE SFH #####
def calculate_sfh(keypoints,mesh_points,mesh_normals, bins=8, r=1):
    """ Function for calculating sfh for each landmark"""
    """
    Calculates the Signed Feature Histogram descriptor given keypoints and mesh surface.
    
    Parameters:
        mesh (vedo mesh): A mesh object representing the vertices and faces of the mesh surface.
        keypoints (numpy array):A list of arrays of shape (1, 3) representing the keypoints of interest.
        radius (float): The radius within which to consider points for the SFH calculations.
        bins (int): The number of bins to use for the histogram (default 8).
    
    Returns:
        sfh_hist (numpy array): An dictionary of arrays of shape (bins, bins, bins) representing the sfh histograms.
    """
    
    #print("Calculating SFH")

    #Initialize full dict
    sfh_dict = {}
    dict_key = 0
    key_count = 0
    tree = cKDTree(mesh_points)
    
    #Calculate sfh for all landmarks
    for i, keypoint in enumerate(keypoints):
        #print("Processing local support: ",round(((i+1)/len(keypoints))*100,2), "%    ", end="\r")
        
        #Initialize matrix for vertex
        vert_mat = []
        
        #Get LRA
        key = -800
        #key = mesh.closestPoint(keypoint,returnPointId=True) 
        #LRAp = mesh.normals()[key]
        dist, indices = tree.query(keypoint, k=len(mesh_points), distance_upper_bound=r)
        indices = indices[~np.isinf(dist)]
        LRAp = np.mean(mesh_normals[indices],axis=0)
        
        #Get tangent plane
        tangent_plane = vedo.Plane(pos=keypoint, normal = LRAp)
        
        #Get local environment to r units
        connected = points_in_radius(mesh_points,keypoint,r=r)

        #Iterate through local environment
        for vert in connected:
            if vert != key:
                
                #Get LRAx
                dist, indices = tree.query(mesh_points[vert], k=len(mesh_points), distance_upper_bound=r)
                indices = indices[~np.isinf(dist)]
                LRAx = np.mean(mesh_normals[indices],axis=0)
                #LRAx = mesh.normals()[vert]
                
                #Get thetax and account for numerical precision
                dot = np.dot(LRAp, LRAx)
                #dot = np.minimum(1, dot)
                #dot = np.maximum(-1, dot)
                Thetax = np.arccos(dot)
                
                #Get D1
                if np.dot(LRAp, (mesh_points[vert]-keypoint)) > 0:
                    D1 =  1
                else:
                    D1 = -1
                
                #Get D2
                if np.dot(LRAp, (mesh_points[vert]-keypoint)) < np.dot(LRAx, (mesh_points[vert]-keypoint)):
                    D2 = 1
                else:
                    D2 = -1
                
                #sfh calculation -> add to matrix
                sfh1 = np.linalg.norm(np.cross((mesh_points[vert]-keypoint),LRAp))
                sfh2 = D1*np.dot((mesh_points[vert]-keypoint),LRAp)
                sfh3 = D2*Thetax
                vert_mat.append([sfh1,sfh2,sfh3])
                
                
        # bin into histogram
        hist_mat, hist_edges = np.histogramdd(np.array(vert_mat),bins=(bins))   #, density=True
                
        #Add matrix to sfh_dict
        sfh_dict[dict_key] = list(hist_mat.flatten())
        dict_key += 1
                
    return sfh_dict

def points_in_radius(mesh,point,r=1):
    """ Function for selecting points within a certain radius"""
    """
    Input: radius, mesh and start point
    Output: list of point IDS
    """
    point = np.array(point)
    mesh_points = np.array(mesh)
    distances = np.linalg.norm(mesh_points-point,axis=1)
    keep_index = np.where(distances < r)[0]
    return keep_index

In [ ]:
def calculate_LRF(mesh, keypoint, radius=1):
    """
    Calculate the Local Reference Frame (LRF) for a given mesh surface, keypoint and radius
    as described in the Rotational Projection Statistics (RoPS) paper.
    
    Parameters:
    mesh (np.ndarray): A numpy array of shape (n, 3) representing the vertices of a mesh surface.
    keypoint (np.ndarray): A numpy array of shape (1, 3) representing the keypoint of interest.
    radius (float): A radius within which to consider points for LRF calculation.
    
    Returns:
    np.ndarray: A numpy array of shape (3, 3) representing the Local Reference Frame (LRF)
    """
    
    # Find all points within radius of the keypoint
    tree = cKDTree(mesh)
    dist, indices = tree.query(keypoint, k=len(mesh), distance_upper_bound=radius)
    indices = indices[~np.isinf(dist)]
    points_within_radius = mesh[indices]
    
    #Calculate weighted points according to distance
    distances = np.linalg.norm(points_within_radius, axis=1)
    weights = 1.0 / (1.0 + distances/radius)
    weighted_points = points_within_radius * weights[:, np.newaxis]

    # Calculate the covariance matrix of the weighted points within radius
    covariance_matrix = np.cov(weighted_points.T)
    
    # Calculate the eigenvectors and eigenvalues of the covariance matrix
    eigenvalues, eigenvectors = np.linalg.eig(covariance_matrix)
    
    # Disambiguation step 
    num_vectors = points_within_radius.shape[0]
    positive_count = np.sum(np.sum(np.sign(points_within_radius) > 0, axis=0))
    negative_count = num_vectors - positive_count
    majority_sign = 1 if positive_count >= negative_count else -1
    eigenvectors = eigenvectors*majority_sign
    
    # Sort the eigenvectors by descending eigenvalues
    sorted_indices = np.argsort(eigenvalues)[::-1]
    sorted_eigenvectors = eigenvectors[:, sorted_indices]
    
    # Use the first two eigenvectors to define the x and y axes of the LRF
    x_axis = sorted_eigenvectors[:, 0]
    z_axis = sorted_eigenvectors[:, 1]
    
    # Take the cross product of the x and y axes to define the z axis of the LRF
    y_axis = np.cross(x_axis, z_axis)
    
    # Construct the LRF as a 3x3 matrix with the axes as columns
    LRF = np.column_stack((x_axis, y_axis, z_axis))
    
    return LRF

In [ ]:
##### CALCULATE ECSAD #####

def cart_to_azel(coord):
    """ Function for converting cartesian coordinates
    to azimuth, elevation and radial coordinates
    """
    x,y,z = coord
    azimuth = np.arctan(x/y)
    elevation = np.arctan(y/z)
    r = np.sqrt(x**2 + y**2 + z**2)
    return np.array([r,azimuth,elevation])

def calculate_surface_normal(normal_vectors):
    # Construct the coefficient matrix A
    A = np.column_stack(normal_vectors)

    # Calculate the pseudo-inverse of A using SVD
    A_pinv = np.linalg.pinv(A)

    # Define the target vector
    b = np.ones((len(normal_vectors),))

    # Solve for the least squares solution x
    x = np.dot(A_pinv, b)

    # Normalize the resulting normal vector
    surface_normal = x / np.linalg.norm(x)

    return surface_normal

def calculate_relative_angle(surface_normal, direction_vector):
    # Normalize the surface normal vector
    surface_normal = surface_normal / np.linalg.norm(surface_normal)

    # Normalize the direction vector
    direction_vector = direction_vector / np.linalg.norm(direction_vector)

    # Calculate the dot product between the surface normal and direction vectors
    dot_product = np.dot(surface_normal, direction_vector)

    # Calculate the relative angle in radians
    relative_angle = np.arccos(dot_product)

    # Convert the relative angle to degrees
    relative_angle_degrees = np.degrees(relative_angle)

    return relative_angle_degrees

def calculate_ECSAD(keypoints,mesh,radius=1):
    """ Function for calculating the Equivalent Circumference Surface Angle Descriptors (ECSAD)
    as descriptors for keypoints"""
    """
    Calculates the Equivalent Circumference Surface Angle Descriptor descriptor given keypoints and mesh surface.
    
    Parameters:
        mesh (vedo mesh): A mesh object representing the vertices and faces of the mesh surface.
        keypoints (numpy array):A list of arrays of shape (1, 3) representing the keypoints of interest.
        radius (float): The radius within which to consider points for the ECSAD calculations.
    
    Returns:
        ecsad_hist (numpy array): An dictionary of arrays of shape (bins, bins, bins) representing the sfh histograms.
    """
    #Initialize mesh and descirptor dict
    mesh_points = mesh.points()
    mesh_norms = mesh.normals()
    ecsad_dict = {}
    dict_key = 0
    saves = []
    #print("Calculating ECSAD")
    
    #Iterate through keypoints
    for key, keypoint in enumerate(keypoints):
        print("Processing keypoints: ",round(((key+1)/len(keypoints))*100,2), "%    ", end="\r")
        
        bin_idx = []
        ecsad = []
        
        # Calculate LRF
        LRF = calculate_LRF(mesh_points, keypoint, radius=radius)
        
        # Get the points within the LRF neighborhood 
        neighborhood_mask = np.sum((mesh_points - keypoint)**2, axis=1) < radius**2
        points_in_neighborhood = mesh_points[neighborhood_mask]
        surf_norm = np.average(mesh_norms[neighborhood_mask])
        neighborhood_idx = np.array(list(range(len(mesh_points))))[neighborhood_mask]
        
        # Move coordinates to origin
        new_origin = points_in_neighborhood-keypoint
        #transform
        points_in_LRF = np.dot(new_origin, LRF.T)
        
        # Project to 2D circle to eliminate elevation
        circle = np.array(new_origin[:,[0,1]])
        
        # Split into 6 (60*) radial angle-bins
        slicenos = np.int32((np.pi + np.arctan2(circle[:, 1], circle[:, 0])) * (6 / (2 * np.pi)))

        # Split each angle-bin into 6 bins according to distance
        r_len = radius/4
        for slicenum in [np.array(0), np.array(1), np.array(2), np.array(3), np.array(4), np.array(5)]:
            idx_copy1 = neighborhood_idx.copy()
            idx1 = np.argwhere(np.isin(slicenos, slicenum)).ravel()
            points_in_slice = circle[idx1]
            idx_copy1 = idx_copy1[idx1]
            distances = np.linalg.norm(points_in_slice, axis=1)
            

            for radial_level in [0,1,2,3]:
                idx_copy2 = idx_copy1.copy()
                idx2 = np.argwhere((distances>=radial_level*r_len)&(distances<=(radial_level+1)*r_len))
                idx2 = idx2.flatten()
                points_in_r_level = points_in_slice[idx2]
                idx_copy2 = idx_copy2[idx2]

                
                # Split each distance bin according to number of bins needed at this distance
                splittings = [(1/(radial_level+1))*split for split in range(0,(radial_level+1))]
                splittings.append(1)
                splittings += slicenum
                bin_val = (np.pi + np.arctan2(points_in_r_level[:, 1], points_in_r_level[:, 0])) * (6 / (2 * np.pi))
                
                for j in range(len(splittings)-1):
                    idx_copy3 = idx_copy2.copy()
                    idx5=np.argwhere((bin_val>splittings[j])&(bin_val<splittings[j+1]))
                    idx5 = idx5.flatten()
                    points_in_bin_level = points_in_r_level[idx5]
                    idx_copy3 = idx_copy3[idx5]
                    bin_idx.append(idx_copy3)
                    
                
        #Get only from bin
        for binn in bin_idx:
            if len(binn) != 0:
                bin_points = mesh_points[binn]
                bin_norms = mesh.normals()[binn]
                dir_vects = bin_points-keypoint
                rel_angle = [calculate_relative_angle(surf_norm, vec) for vec in dir_vects]
                rel_angle = np.average(rel_angle)
            else:
                rel_angle = None
                
            ecsad.append(rel_angle)

        #Interpolation
        #Inner circle
        ecsad_copy1 = ecsad.copy()
        first_circle = [0,10,20,30,40,50]
        second_circle = [1,2,11,12,21,22,31,32,41,42,51,52]
        third_circle = [3,4,5,13,14,15,23,24,25,33,34,35,43,44,45,53,54,55]
        fourth_circle = [6,7,8,9,16,17,18,19,26,27,28,29,36,37,38,39,
                        46,47,48,49,56,57,58,59]
        for i,t in enumerate(first_circle):
            if ecsad[t] == None:
                lower = [0]
                middle = []
                higher = []
                if t+10 < len(ecsad):
                    j = t+10
                else:
                    j = 0
                if ecsad[j] != None:
                    middle.append(ecsad[j])
                if ecsad[t-10] != None:
                    middle.append(ecsad[t-10])
                    
                if t+1 < len(ecsad):
                    j=t+1
                else:
                    j=0
                if ecsad[j] != None:
                    higher.append(ecsad[j])
                if t+2 < len(ecsad):
                    j=t+2
                else:
                    j=(t+2)-len(ecsad)
                if ecsad[j] != None:
                    higher.append(ecsad[j])
                          
                new_val = np.average(np.concatenate((lower,middle,higher)).flatten())
                ecsad_copy1[t] = new_val
                
        
        #Second circle
        ecsad_copy2 = ecsad_copy1.copy()
        for i,t in enumerate(second_circle):
            if ecsad_copy1[t] == None:
                if int(str(t)[-1]) == 2:
                    j = t-2
                else:
                    j = t-1
                
                if ecsad_copy1[j] != None:
                    lower = [ecsad_copy1[j]]
                else:
                    lower = []
                middle = []
                higher = []
                if  i+1 < len(second_circle):
                    j = i+1
                else:
                    j= 0
                if ecsad_copy1[second_circle[j]] != None: 
                    middle.append(ecsad_copy1[second_circle[j]])
                if ecsad_copy1[second_circle[i-1]] != None:
                    middle.append(ecsad_copy1[second_circle[i-1]])
                if t+2 < len(ecsad_copy1):
                    j=t+2
                else:
                    j=(t+2)-len(ecsad_copy1)
                if ecsad_copy1[j] != None:
                    higher.append(ecsad_copy1[t+2])
                if t+3 < len(ecsad_copy1):
                    j=t+3
                else:
                    j=(t+3)-len(ecsad_copy1)
                if ecsad_copy1[j] != None:
                    higher.append(ecsad_copy1[j])
                    
                new_val = np.average(np.concatenate((lower,middle,higher)).flatten())
                ecsad_copy2[t] = new_val
                
        
        #third circle
        ecsad_copy3 = ecsad_copy2.copy()
        for i,t in enumerate(third_circle):
        
            
            if ecsad_copy2[t] == None:
                if int(str(t)[-1]) == 5:
                    j = t-3
                else :
                    j = t -2
       
                if ecsad_copy2[j] != None:
                    lower = [ecsad_copy2[j]]
                else:
                    lower =  []
                middle = []
                higher = []
                if i+1 == len(third_circle):
                    jr = third_circle[0]
                else:
                    jr = third_circle[i+1]
                jl = third_circle[i-1]
                if ecsad_copy2[jr] != None:
                    middle.append(ecsad_copy2[jr])
                if ecsad_copy2[jl] != None:
                    middle.append(ecsad_copy2[jl])
                if t+3 < len(ecsad_copy2):
                    j=t+3
                else:
                    j=(t+3)-len(ecsad_copy2)
                if ecsad_copy2[j] != None:
                    higher.append(ecsad_copy2[j])
                if t+4 < len(ecsad_copy2):
                    j=t+4
                else:
                    j=(t+4)-len(ecsad_copy2)
                if ecsad_copy2[j] != None:
                    higher.append(ecsad_copy2[j])
                   
                new_val = np.average(np.concatenate((lower,middle,higher)).flatten())
                ecsad_copy3[t] = new_val
                
                
        #fourth circle
        ecsad_copy4 = ecsad_copy3.copy()
        for i,t in enumerate(fourth_circle):
            if ecsad_copy3[t] == None:
                if int(str(t)[-1]) == 9:
                    j = t-4
                else :
                    j = t -3
       
                if ecsad_copy3[j] != None:
                    lower = [ecsad_copy3[j]]
                else:
                    lower =  []
                middle = []
                higher = []
                if i+1 == len(fourth_circle):
                    jr = fourth_circle[0]
                else:
                    jr = fourth_circle[i+1]
                jl = fourth_circle[i-1]
                if ecsad_copy3[jr] != None:
                    middle.append(ecsad_copy3[jr])
                if ecsad_copy3[jl] != None:
                    middle.append(ecsad_copy3[jl])
                   
                new_val = np.average(np.concatenate((lower,middle,higher)).flatten())
                ecsad_copy4[t] = new_val
             
                 
        
        #Summing opposing bins
        ecsad_out = []
        for i in range(30):
            ecsad_out.append(ecsad_copy4[i]+ecsad_copy4[i+30])


        ecsad_dict[dict_key]=list(np.array(ecsad_out))
        dict_key += 1



        

    return ecsad_dict
  
        

In [ ]:
###### CALCULATE ROPS #####

def central_moment(D,m,n):
    """ Central moment of 2D dsitribution matrix D, in order m and n"""
    meanX = np.sum(D, axis=0) @ np.arange(1, D.shape[1]+1)
    meanY = np.sum(D, axis=1) @ np.arange(1, D.shape[0]+1)
    X, Y = np.meshgrid(np.arange(1, D.shape[1]+1), np.arange(1, D.shape[0]+1))
    integrandXY_mn = np.power((X - meanX),m) * np.power((Y - meanY),n) * D
    momentXY_mn = np.sum(integrandXY_mn)
    return momentXY_mn


def shannon_entropy(matrix):
    """
    Calculates the Shannon entropy of a 2D matrix.

    Parameters:
        matrix (numpy array): A 2D array.

    Returns:
        entropy (float): The Shannon entropy of the matrix.
    """

    # Normalize the matrix
    matrix = matrix / np.sum(matrix)

    # Calculate the entropy
    mask = matrix > 0  # Filter out zeros or negative values
    entropy = -np.sum(matrix[mask] * np.log2(matrix[mask] + np.finfo(float).eps))

    return np.float64(entropy)

def calculate_RoPS(mesh, keypoint, LRF, radius=1, num_bins=5, num_angles=3):
    """
    Calculates the Rotational Projection Statistics (RoPS) descriptor for a given keypoint and mesh surface.
    
    Parameters:
        mesh (numpy array): An array of shape (n, 3) representing the vertices of the mesh surface.
        keypoint (numpy array): An array of shape (1, 3) representing the keypoint of interest.
        LRF (numpy array): An array of shape (3, 3) representing the local reference frame at the keypoint.
        radius (float): The radius within which to consider points for the LRF and RoPS calculations.
        num_bins (int): The number of bins to use for the distance and elevation axes (default 3).
        num_angles (int): The number of angles to use for the azimuth axis (default 15).
    
    Returns:
        RoPS_hist (numpy array): An array of shape (num_bins, num_bins, num_angles) representing the RoPS histogram.
    """
    
    # Define the angle increments
    angle_increments = np.linspace(0, 2*np.pi, num_angles+1)[:-1]

    
    # Initialize the RoPS histogram
    RoPS_hist = np.zeros((num_angles, 3 , 15))


    ### Get Local Surface

    # Get the keypoint in the LRF coordinates 
    keypoint_in_LRF = np.dot(keypoint - keypoint, LRF.T)

    # Get the points within the LRF neighborhood and transform
    neighborhood_mask = np.sum((mesh - keypoint)**2, axis=1) < radius**2
    points_in_neighborhood = mesh[neighborhood_mask]
    points_in_LRF = np.dot(points_in_neighborhood, LRF.T)
    
    ### Get Rotated surfaces
    projections = []
    #Define rotation matrices
    for theta in angle_increments:
        rot_mat_xyz = [[[1,0,0],[0,np.cos(theta),-np.sin(theta)],[0, np.sin(theta), np.cos(theta)]],
                      [[np.cos(theta),0 ,np.sin(theta)],[0,1,0],[-np.sin(theta), 0, np.cos(theta)]],
                      [[np.cos(theta),-np.sin(theta), 0],[np.sin(theta), np.cos(theta), 0],[0,0,1]]]
            
        # Rotate the point cloud
        rotated_surfaces = np.array([np.array(list(map(np.matmul,[rot_mat_xyz[rot_i]]*len(points_in_LRF),points_in_LRF))) for rot_i in [0,1,2]])

        # Project the pointcloud onto the axis
        P1 = rotated_surfaces[:,:,[0,1]]
        P2 = rotated_surfaces[:,:,[0,2]]
        P3 = rotated_surfaces[:,:,[1,2]]
        projections.append(P1)
        projections.append(P2)
        projections.append(P3)

    # Get histogram for each projection, for each rotation
    hist_mats = np.array([np.histogramdd(np.array(projection_sub), bins=(num_bins))[0] for projection in projections for projection_sub in projection])

    # Normalize each histogram in hist_mats (without the risk of complex numbers)
    normalized_hist_mats = []
    for hist_mat in hist_mats:
        total_count = np.sum(hist_mat)
        if total_count != 0:
            normalized_hist_mat = hist_mat / total_count
            normalized_hist_mats.append(normalized_hist_mat)
        else:
            # Handle the case where total_count is zero to avoid division by zero
            normalized_hist_mats.append(hist_mat)

    # Convert the list of normalized histograms back to an array
    normalized_hist_mats = np.array(normalized_hist_mats)
    
    
    ### Get Projection Statistics
    entropies = np.array([list(map(shannon_entropy, hist_mats))]).T
    momentums = np.array([[central_moment(D,mom_i,mom_j) for mom_i in [1,2] for mom_j in [1,2]] for D in hist_mats])
    descriptors = np.concatenate((entropies,momentums),axis=1)
    descriptor = descriptors.flatten()
    #descriptor = descriptor / np.linalg.norm(descriptor) #normalize
    
    return list(descriptor)

def RoPS_Estimator(keypoints,mesh, L=5, T=3, r=15):
    """ Function for calculating RoPS for each landmark
    from paper: https://link.springer.com/article/10.1007/s11263-013-0627-y?utm_source=getftr&utm_medium=getftr&utm_campaign=getftr_pilot
    """
    """
    Input: landmark coordinates and mesh
    Output: RoPS for each landmark
    """
    
    #print("Calculating RoPS                  ")
    mesh = mesh.points()

    #Initialize full dict
    rops_dict = {}
    dict_key = 0
    
    #Calculate rops for all landmarks
    for i, keypoint in enumerate(keypoints):
        keypoint = np.reshape(keypoint, (-1, 3))
        #print("Processing keypoints: ",round(((i+1)/len(keypoints))*100,2), "%    ", end="\r")
        
        
        LRF = calculate_LRF(keypoint, mesh, radius=r)
        rops = calculate_RoPS(mesh, keypoint, LRF, radius=r, num_bins=L, num_angles=T)
        rops_dict[dict_key] = rops
        dict_key += 1
        
    return rops_dict

In [ ]:
##### CALCULATE USC #####
def cart_to_azel(coord):
    """ Function for converting cartesian coordinates
    to azimuth, elevation and radial coordinates
    """
    """
    Input: list with x, y, and z coordinate in cartesian system
    Output: list of radial, azimuth and elevation coordinates
    """
    x,y,z = coord
    azimuth = np.arctan(x/y)
    elevation = np.arctan(y/z)
    r = np.sqrt(x**2 + y**2 + z**2)
    return np.array([r,azimuth,elevation])

def calculate_USC(keypoints, mesh, radius = 1, n_azi=12, n_radi=15, n_elev=11):
    """ Function for calculating the Unique Shape Context (USC)
    as descriptors for keypoints"""
    """
    Calculates the Unique Shape Context descriptor given keypoints, radius, bins, and mesh surface.
    
    Parameters:
        mesh (vedo mesh): A mesh object representing the vertices and faces of the mesh surface.
        keypoints (numpy array):A list of arrays of shape (1, 3) representing the keypoints of interest.
        radius (float): The radius within which to consider points for the SFH calculations.
        bins (int): The number of bins to use for the respective directions.
    
    Returns:
        ush_hist (numpy array): An dictionary of arrays of shape (bins, bins, bins) representing the sfh histograms.
    """
    #Initialize mesh and descirptor dict
    #print("Calculating USC descriptors")
    mesh = mesh.points()
    usc_dict = {}
    dict_key = 0
    
    #Iterate through keypoints
    for key, keypoint in enumerate(keypoints):
        #print("Processing local support: ",round(((key+1)/len(keypoints))*100,2), "%    ", end="\r")
        
        # Calculate LRF
        LRF = calculate_LRF(mesh, keypoint, radius=radius)
        
        # Get the points within the LRF neighborhood and transform
        neighborhood_mask = np.sum((mesh - keypoint)**2, axis=1) < radius**2
        points_in_neighborhood = mesh[neighborhood_mask]
        points_in_LRF = np.dot(points_in_neighborhood, LRF.T)
        
        # From cartesian to azimuth, elevation and radial coordinates
        azel_coord = np.array([list(map(cart_to_azel, points_in_LRF))])
        azel_coord = np.squeeze(azel_coord)
        
        #Calculate spherical histogram
        hist_mat, hist_edges = np.histogramdd(np.array(azel_coord),bins=(n_radi,n_azi,n_elev))   #, density=True
        
        #Add matrix to usc_dict
        usc_dict[dict_key] = list(hist_mat.flatten())
        dict_key += 1
        
        
    return usc_dict

In [ ]:
##### CALCULATE SHOT #####
def calculate_SHOT(keypoints,mesh, radius=1):
    """ Function for calculating SHOT descriptor
    NOTICE: the keypoints should be given as point idx
    Also notice the usage of the pyshot package
    """
    """
    Calculates the  Signature of Histograms of OrienTations descriptor given keypoints and mesh surface.
    
    Parameters:
        mesh (vedo mesh): A mesh object representing the vertices and faces of the mesh surface.
        keypoints (numpy array):A list of point indexes representing the keypoints of interest.
        radius (float): The radius within which to consider points for the SHOT calculations.
    
    Returns:
        shot_hist (numpy array): An dictionary of arrays representing the sfh histograms.
    """
    #print("Calculating Shot descriptors                                  ")
    vertices: np.array =  np.array(mesh.points().astype(np.float64))
    faces: np.array =  np.array(mesh.faces())

    # a np.array of shape (n, n_descr)
    shot_descrs: np.array = pyshot.get_descriptors(
        vertices,
        faces,
        radius=radius,
        local_rf_radius=radius,
        # The following parameters are optional
        min_neighbors=3,
        n_bins=10,
        double_volumes_sectors=True,
        use_interpolation=True,
        use_normalization=True,
        )
    keypoint_descrs = shot_descrs[keypoints]
    #keypoint_descrs = np.array([list(map(normalize,keypoint_descrs))])  #normalize
    keypoint_descrs = dict(enumerate(keypoint_descrs))
    keypoint_descrs = {dict_key:list(v) for dict_key, (k, v) in enumerate(keypoint_descrs.items())}

    return keypoint_descrs

In [ ]:
def run_shot_single(path,query,q_mesh,r):
    q_rep = calculate_SHOT(query,q_mesh, radius=r)
    print("Number of keypoints: ",len(q_rep))
    with open(path, "w") as fp:
        json.dump(q_rep , fp) 

In [ ]:
def run_usc_single(path,query,q_mesh,r):
    q_rep = calculate_USC(query,q_mesh, radius=r)
    print("Number of keypoints: ",len(q_rep))
    with open(path, "w") as fp:
        json.dump(q_rep , fp) 

In [ ]:
def run_rops_single(path,query,q_mesh,r):
    q_rep = RoPS_Estimator(query,q_mesh, r=r)
    print("Number of keypoints: ",len(q_rep))
    with open(path, "w") as fp:
        json.dump(q_rep , fp) 

In [ ]:
def run_ecsad_single(path,query,q_mesh,r):
    q_rep = calculate_ECSAD(query,q_mesh, radius=r)
    print("Number of keypoints: ",len(q_rep))
    with open(path, "w") as fp:
        json.dump(q_rep , fp) 

In [ ]:
def run_sfh_single(path,query,q_mesh,r,bins=8):
    q_rep = calculate_sfh(query,q_mesh.points(),q_mesh.normals(), bins=bins, r=r)
    print("Number of keypoints: ",len(q_rep))
    with open(path, "w") as fp:
        json.dump(q_rep , fp) 

## Calculate representation

#### For single methods

In [ ]:
res = 40
for method in ["rops","usc","ecsad","shot","sfh"]: #["rops","usc","ecsad","shot","sfh"]
    print(f"Working with method {method}")
    inspection = False
    if method == "sfh":
        run_single = run_sfh_single
        kp_id = False
        kp_pts = True
        bins = 8
    elif method == "ecsad":
        run_single = run_ecsad_single
        kp_id = False
        kp_pts = True
    elif method == "rops":
        run_single = run_rops_single
        kp_id = False
        kp_pts = True
        L = 5
        T = 3
    elif method == "usc":
        run_single = run_usc_single
        kp_id = False
        kp_pts = True
    elif method == "shot":
        run_single = run_shot_single
        kp_id = True
        kp_pts = False
    
    
    # Get all combinations of dental input
    in_path = "/path/to/nput/path/"
    names = os.listdir(in_path)
    inspection = False
    
    if method not in []:
        for r in [2]: #[1,1.5,2]
            if method == "sfh":
                method_name = method +f"_r{r}_bins{bins}"
            else:
                method_name = method +f"_r{r}"
            out_path = f"Path/to/output/path/{method_name}/"
            os.mkdir(out_path)
    
            for kp_num, name in enumerate(names):
                print(name)
                print(f"Keypoint detection: {round((kp_num/len(names))*100,2)}%        1/8", end = "\r")
            
                # Keypoint detection for full dentition
                teeth_full = vedo.load(os.path.join(in_path,name))
                kp = kpd.keypoint_detection(teeth_full, res=res, name=f"{name[4:28]}_full", returnIdx = kp_id, returnPts=kp_pts,output=out_path, inspection = inspection)
                q_path = os.path.join(out_path,f'{name.split(".")[0]}_full.json')
                run_single(q_path,kp,teeth_full,r)        
                print(f"Keypoint detection: {round((kp_num/len(names))*100,2)}%        2/8", end = "\r")
            
                # Keypoint detection for noisy dentition (0.1)
                mel = 1/round(len(teeth_full.points())/teeth_full.area(),3)
                teeth_n1 = teeth_full.clone()
                n1 = 0.1*mel
                teeth_n1 = teeth_n1.pointGaussNoise(sigma=n1)
                kp_n1 = kpd.keypoint_detection(teeth_n1, res=res, name=f"{name[4:28]}_full_n1", returnIdx = kp_id, returnPts=kp_pts,output=out_path, inspection = inspection)
                q_path = os.path.join(out_path,f'{name.split(".")[0]}_full_n1.json')
                run_single(q_path,kp_n1,teeth_n1,r) 
                print(f"Keypoint detection: {round((kp_num/len(names))*100,2)}%        3/8", end = "\r")
            
                # Keypoint detection for noisy dentition (0.3)            
                teeth_n3 = teeth_full.clone()
                n3 = 0.3*mel
                teeth_n3 = teeth_n1.pointGaussNoise(sigma=n3)
                kp_n3 = kpd.keypoint_detection(teeth_n3, res=res, name=f"{name[4:28]}_full_n3", returnIdx = kp_id, returnPts=kp_pts,output=out_path, inspection = inspection)
                q_path = os.path.join(out_path,f'{name.split(".")[0]}_full_n3.json')
                run_single(q_path,kp_n3,teeth_n3,r)             
                print(f"Keypoint detection: {round((kp_num/len(names))*100,2)}%        4/8", end = "\r")
            
                #keypoint detection for partial 1
                teeth_partial1 = teeth_full.clone()
                for i, point in enumerate(teeth_full.points()):
                    teeth_partial1.points()[i] = point + np.array([100,0,0])
                teeth_partial1 = teeth_partial1.cutWithPlane(origin= teeth_partial1.centerOfMass(), normal = (1,0,0))
                teeth_partial1.decimate(fraction=0.9 ,method='quadratic')
                kp1a = kpd.keypoint_detection(teeth_partial1, res=res, name=f"{name[4:29]}_partial1", returnIdx = kp_id, returnPts=kp_pts, output=out_path, inspection = inspection)
                q_path = os.path.join(out_path,f'{name.split(".")[0]}_part1.json')
                run_single(q_path,kp1a,teeth_partial1,r) 
                print(f"Keypoint detection: {round((kp_num/len(names))*100,2)}%        5/8", end = "\r")
            
                #keypoint detection for partial 2
                teeth_partial2 = teeth_full.clone()
                for i, point in enumerate(teeth_full.points()):
                    teeth_partial2.points()[i] = point + np.array([0,-100,0])
                teeth_partial2 = teeth_partial2.cutWithPlane(origin= teeth_partial2.centerOfMass(), normal = (0,1,0))
                teeth_partial2.decimate(fraction=0.85 ,method='quadratic')
                kp1b = kpd.keypoint_detection(teeth_partial2, res=res, name=f"{name[4:29]}_partial2", returnIdx = kp_id,returnPts=kp_pts, output=out_path, inspection = inspection)
                q_path = os.path.join(out_path,f'{name.split(".")[0]}_part2.json')
                run_single(q_path,kp1b,teeth_partial2,r) 
                print(f"Keypoint detection: {round((kp_num/len(names))*100,2)}%        6/8", end = "\r")
            
                # Keypoint detection for noisy dentition (0.1)
                mel = 1/round(len(teeth_partial2.points())/teeth_partial2.area(),3)
                teeth_part_n1 = teeth_partial2.clone()
                n1 = 0.1*mel
                teeth_part_n1 = teeth_part_n1.pointGaussNoise(sigma=n1)
                kp1b_n1 = kpd.keypoint_detection(teeth_part_n1, res=res, name=f"{name[4:28]}_partial2_n1", returnIdx = kp_id, returnPts=kp_pts,output=out_path, inspection = inspection)
                q_path = os.path.join(out_path,f'{name.split(".")[0]}_part2_n1.json')
                run_single(q_path,kp1b_n1,teeth_part_n1,r) 
                print(f"Keypoint detection: {round((kp_num/len(names))*100,2)}%        7/8", end = "\r")
            
                # Keypoint detection for noisy dentition (0.1)
                mel = 1/round(len(teeth_partial2.points())/teeth_partial2.area(),3)
                teeth_part_n3 = teeth_partial2.clone()
                n3 = 0.3*mel
                teeth_part_n3 = teeth_part_n3.pointGaussNoise(sigma=n3)
                kp1b_n3 = kpd.keypoint_detection(teeth_part_n3, res=res, name=f"{name[4:28]}_partial2_n3", returnIdx = kp_id, returnPts=kp_pts,output=out_path, inspection = inspection)
                q_path = os.path.join(out_path,f'{name.split(".")[0]}_part2_n3.json')
                run_single(q_path,kp1b_n3,teeth_part_n3,r) 
                print(f"Keypoint detection: {round((kp_num/len(names))*100,2)}%        8/8", end = "\r")
            
                #keypoint detection for partial 3
                teeth_partial3 = teeth_full.clone()
                theta=0.3
                mat = [[np.cos(theta),-np.sin(theta),0],[np.sin(theta), np.cos(theta), 0],[0,0,1]]
                for i, point in enumerate(teeth_full.points()):
                    teeth_partial3.points()[i] = np.matmul(mat,point)
                teeth_partial3.decimate(fraction=0.80 ,method='quadratic')
                kp1c = kpd.keypoint_detection(teeth_partial3, res=res, name=f"{name[4:29]}_partial3", returnIdx = kp_id,returnPts=kp_pts, output=out_path, inspection = inspection)
                q_path = os.path.join(out_path,f'{name.split(".")[0]}_rot.json')
                run_single(q_path,kp1c,teeth_partial3,r) 
                print(f"Keypoint detection: {round((kp_num/len(names))*100,2)}%        8/8", end = "\r")

#### For combination methods

In [ ]:
def Normalize_Data(data):
    data = np.array(data)
    norm_data = (data - np.min(data)) / (np.max(data) - np.min(data))
    return list(norm_data)

In [ ]:
def run_sfh_single(path,query,q_mesh,r,bins=6):
    q_rep = calculate_sfh(query,q_mesh.points(),q_mesh.normals(), bins=bins, r=r)
    print("Number of keypoints: ",len(q_rep))
    return q_rep 

In [ ]:
def run_ecsad_single(path,query,q_mesh,r):
    q_rep = calculate_ECSAD(query,q_mesh, radius=r)
    return q_rep 

In [ ]:
def run_rops_single(path,query,q_mesh,r):
    q_rep = RoPS_Estimator(query,q_mesh, r=r)
    print("Number of keypoints: ",len(q_rep))
    return q_rep 

In [ ]:
def run_usc_single(path,query,q_mesh,r):
    q_rep = calculate_USC(query,q_mesh, radius=r)
    print("Number of keypoints: ",len(q_rep))
    return q_rep 

In [ ]:
def run_shot_single(path,query,q_mesh,r):
    q_rep = calculate_SHOT(query,q_mesh, radius=r)
    print("Number of keypoints: ",len(q_rep))
    return q_rep 

In [ ]:
res = 30
for method in [["sfh","shot"], ["sfh","ecsad"]]: #["rops","usc","ecsad","shot","sfh"] , #["sfh","shot"], ["sfh","ecsad"],["sfh","usc"],["shot","usc"],["shot","ecsad"],["ecsad","usc"]
    print(f"Working with method {method}")
    inspection = False
    if method[0] == "sfh":
        run_single1 = run_sfh_single
        kp_id1 = False
        kp_pts1 = True
        bins = 6
    elif method[0] == "ecsad":
        run_single1 = run_ecsad_single
        kp_id1 = False
        kp_pts1 = True
    elif method[0] == "rops":
        run_single1 = run_rops_single
        kp_id1 = False
        kp_pts1 = True
        L = 5
        T = 3
    elif method[0] == "usc":
        run_single1 = run_usc_single
        kp_id1 = False
        kp_pts1 = True
    elif method[0] == "shot":
        run_single1 = run_shot_single
        kp_id1 = True
        kp_pts1 = False
        
    if method[1] == "sfh":
        run_single2 = run_sfh_single
        kp_id2 = False
        kp_pts2 = True
        bins = 6
    elif method[1] == "ecsad":
        run_single2 = run_ecsad_single
        kp_id2 = False
        kp_pts2 = True
    elif method[1] == "rops":
        run_single2 = run_rops_single
        kp_id2 = False
        kp_pts2 = True
        L = 5
        T = 3
    elif method[1] == "usc":
        run_single2 = run_usc_single
        kp_id2 = False
        kp_pts2 = True
    elif method[1] == "shot":
        run_single2 = run_shot_single
        kp_id2 = True
        kp_pts2 = False
    
    
    # Get all combinations of dental input
    in_path = "/path/to/input/path/"
    names = os.listdir(in_path)
    inspection = False
    
    if method not in []:
        for r in [2]: 
            method_name =  method[0] +"_"+method[1] + "_"
            if "sfh" in method:
                method_name += f"r{r}_bins{bins}"
            else:
                method_name += f"r{r}"
            out_path = f"/path/to/output/path/{method_name}/"
            os.mkdir(out_path)
    
            for kp_num, name in enumerate(names):
                print(name)
                print(f"Keypoint detection: {round((kp_num/len(names))*100,2)}%        1/8", end = "\r")
            
                # Keypoint detection for full dentition
                teeth_full = vedo.load(os.path.join(in_path,name))
                if kp_id1 == kp_id2:
                    kp_id = kp_id1
                    kp_pts = kp_pts1
                    kp = kpd.keypoint_detection(teeth_full, res=res, name=f"{name[4:28]}_full", returnIdx = kp_id, returnPts=kp_pts,output=out_path, inspection = inspection)
                    q_path = os.path.join(out_path,f'{name.split(".")[0]}_full.json')
                    q1 = run_single1(q_path,kp,teeth_full,r)
                    q2 = run_single2(q_path,kp,teeth_full,r)
                else:
                    kp1 = kpd.keypoint_detection(teeth_full, res=res, name=f"{name[4:28]}_full", returnIdx = kp_id1, returnPts=kp_pts1,output=out_path, inspection = inspection)
                    kp2 = kpd.keypoint_detection(teeth_full, res=res, name=f"{name[4:28]}_full", returnIdx = kp_id2, returnPts=kp_pts2,output=out_path, inspection = inspection)
                    q_path = os.path.join(out_path,f'{name.split(".")[0]}_full.json')
                    q1 = run_single1(q_path,kp1,teeth_full,r)
                    q2 = run_single2(q_path,kp2,teeth_full,r)
                q3 = {}
                for k,v in q1.items():
                    q3[k] = list(np.concatenate([Normalize_Data(v),Normalize_Data(q2[k])]))
                with open(q_path, "w") as fp:
                    json.dump(q3 , fp)
                print(f"Keypoint detection: {round((kp_num/len(names))*100,2)}%        2/8", end = "\r")
            
                # Keypoint detection for noisy dentition (0.1)
                mel = 1/round(len(teeth_full.points())/teeth_full.area(),3)
                teeth_n1 = teeth_full.clone()
                n1 = 0.1*mel
                teeth_n1 = teeth_n1.pointGaussNoise(sigma=n1)
                if kp_id1 == kp_id2:
                    kp_id = kp_id1
                    kp_pts = kp_pts1
                    kp = kpd.keypoint_detection(teeth_n1, res=res, name=f"{name[4:28]}_full_n1", returnIdx = kp_id, returnPts=kp_pts,output=out_path, inspection = inspection)
                    q_path = os.path.join(out_path,f'{name.split(".")[0]}_full_n1.json')
                    q1 = run_single1(q_path,kp,teeth_full,r)
                    q2 = run_single2(q_path,kp,teeth_full,r)
                else:
                    kp1 = kpd.keypoint_detection(teeth_n1, res=res, name=f"{name[4:28]}_full_n1", returnIdx = kp_id1, returnPts=kp_pts1,output=out_path, inspection = inspection)
                    kp2 = kpd.keypoint_detection(teeth_n1, res=res, name=f"{name[4:28]}_full_n1", returnIdx = kp_id2, returnPts=kp_pts2,output=out_path, inspection = inspection)
                    q_path = os.path.join(out_path,f'{name.split(".")[0]}_full_n1.json')
                    q1 = run_single1(q_path,kp1,teeth_n1,r)
                    q2 = run_single2(q_path,kp2,teeth_n1,r)
                q3 = {}
                for k,v in q1.items():
                    q3[k] = list(np.concatenate([Normalize_Data(v),Normalize_Data(q2[k])]))
                with open(q_path, "w") as fp:
                    json.dump(q3 , fp)
                print(f"Keypoint detection: {round((kp_num/len(names))*100,2)}%        3/8", end = "\r")
            
                # Keypoint detection for noisy dentition (0.3)            
                teeth_n3 = teeth_full.clone()
                n3 = 0.3*mel
                teeth_n3 = teeth_n1.pointGaussNoise(sigma=n3)
                if kp_id1 == kp_id2:
                    kp_id = kp_id1
                    kp_pts = kp_pts1
                    kp = kpd.keypoint_detection(teeth_n3, res=res, name=f"{name[4:28]}_full_n3", returnIdx = kp_id, returnPts=kp_pts,output=out_path, inspection = inspection)
                    q_path = os.path.join(out_path,f'{name.split(".")[0]}_full_n3.json')
                    q1 = run_single1(q_path,kp,teeth_n3,r)
                    q2 = run_single2(q_path,kp,teeth_n3,r)
                else:
                    kp1 = kpd.keypoint_detection(teeth_n3, res=res, name=f"{name[4:28]}_full_n3", returnIdx = kp_id1, returnPts=kp_pts1,output=out_path, inspection = inspection)
                    kp2 = kpd.keypoint_detection(teeth_n3, res=res, name=f"{name[4:28]}_full_n3", returnIdx = kp_id2, returnPts=kp_pts2,output=out_path, inspection = inspection)
                    q_path = os.path.join(out_path,f'{name.split(".")[0]}_full_n3.json')
                    q1 = run_single1(q_path,kp1,teeth_n3,r)
                    q2 = run_single2(q_path,kp2,teeth_n3,r)
                q3 = {}
                for k,v in q1.items():
                    q3[k] = list(np.concatenate([Normalize_Data(v),Normalize_Data(q2[k])]))
                with open(q_path, "w") as fp:
                    json.dump(q3 , fp)             
                print(f"Keypoint detection: {round((kp_num/len(names))*100,2)}%        4/8", end = "\r")
            
                #keypoint detection for partial 1
                teeth_partial1 = teeth_full.clone()
                for i, point in enumerate(teeth_full.points()):
                    teeth_partial1.points()[i] = point + np.array([100,0,0])
                teeth_partial1 = teeth_partial1.cutWithPlane(origin= teeth_partial1.centerOfMass(), normal = (1,0,0))
                teeth_partial1.decimate(fraction=0.9 ,method='quadratic')
                if kp_id1 == kp_id2:
                    kp_id = kp_id1
                    kp_pts = kp_pts1
                    kp = kpd.keypoint_detection(teeth_partial1, res=res, name=f"{name[4:28]}_part1", returnIdx = kp_id, returnPts=kp_pts,output=out_path, inspection = inspection)
                    q_path = os.path.join(out_path,f'{name.split(".")[0]}_part1.json')
                    q1 = run_single1(q_path,kp,teeth_partial1,r)
                    q2 = run_single2(q_path,kp,teeth_partial1,r)
                else:
                    kp1 = kpd.keypoint_detection(teeth_partial1, res=res, name=f"{name[4:28]}_part1", returnIdx = kp_id1, returnPts=kp_pts1,output=out_path, inspection = inspection)
                    kp2 = kpd.keypoint_detection(teeth_partial1, res=res, name=f"{name[4:28]}_part1", returnIdx = kp_id2, returnPts=kp_pts2,output=out_path, inspection = inspection)
                    q_path = os.path.join(out_path,f'{name.split(".")[0]}_part1.json')
                    q1 = run_single1(q_path,kp1,teeth_partial1,r)
                    q2 = run_single2(q_path,kp2,teeth_partial1,r)
                q3 = {}
                for k,v in q1.items():
                    q3[k] = list(np.concatenate([Normalize_Data(v),Normalize_Data(q2[k])]))
                with open(q_path, "w") as fp:
                    json.dump(q3 , fp)
                print(f"Keypoint detection: {round((kp_num/len(names))*100,2)}%        5/8", end = "\r")
            
                #keypoint detection for partial 2
                teeth_partial2 = teeth_full.clone()
                for i, point in enumerate(teeth_full.points()):
                    teeth_partial2.points()[i] = point + np.array([0,-100,0])
                teeth_partial2 = teeth_partial2.cutWithPlane(origin= teeth_partial2.centerOfMass(), normal = (0,1,0))
                teeth_partial2.decimate(fraction=0.85 ,method='quadratic')
                if kp_id1 == kp_id2:
                    kp_id = kp_id1
                    kp_pts = kp_pts1
                    kp = kpd.keypoint_detection(teeth_partial2, res=res, name=f"{name[4:28]}_part2", returnIdx = kp_id, returnPts=kp_pts,output=out_path, inspection = inspection)
                    q_path = os.path.join(out_path,f'{name.split(".")[0]}_part2.json')
                    q1 = run_single1(q_path,kp,teeth_partial2,r)
                    q2 = run_single2(q_path,kp,teeth_partial2,r)
                else:
                    kp1 = kpd.keypoint_detection(teeth_partial2, res=res, name=f"{name[4:28]}_part2", returnIdx = kp_id1, returnPts=kp_pts1,output=out_path, inspection = inspection)
                    kp2 = kpd.keypoint_detection(teeth_partial2, res=res, name=f"{name[4:28]}_part2", returnIdx = kp_id2, returnPts=kp_pts2,output=out_path, inspection = inspection)
                    q_path = os.path.join(out_path,f'{name.split(".")[0]}_part2.json')
                    q1 = run_single1(q_path,kp1,teeth_partial2,r)
                    q2 = run_single2(q_path,kp2,teeth_partial2,r)
                q3 = {}
                for k,v in q1.items():
                    q3[k] = list(np.concatenate([Normalize_Data(v),Normalize_Data(q2[k])]))
                with open(q_path, "w") as fp:
                    json.dump(q3 , fp)
                print(f"Keypoint detection: {round((kp_num/len(names))*100,2)}%        6/8", end = "\r")
            
                # Keypoint detection for noisy dentition (0.1)
                mel = 1/round(len(teeth_partial2.points())/teeth_partial2.area(),3)
                teeth_part_n1 = teeth_partial2.clone()
                n1 = 0.1*mel
                teeth_part_n1 = teeth_part_n1.pointGaussNoise(sigma=n1)
                if kp_id1 == kp_id2:
                    kp_id = kp_id1
                    kp_pts = kp_pts1
                    kp = kpd.keypoint_detection(teeth_part_n1, res=res, name=f"{name[4:28]}_part_n1", returnIdx = kp_id, returnPts=kp_pts,output=out_path, inspection = inspection)
                    q_path = os.path.join(out_path,f'{name.split(".")[0]}_part_n1.json')
                    q1 = run_single1(q_path,kp,teeth_part_n1,r)
                    q2 = run_single2(q_path,kp,teeth_part_n1,r)
                else:
                    kp1 = kpd.keypoint_detection(teeth_part_n1, res=res, name=f"{name[4:28]}_part_n1", returnIdx = kp_id1, returnPts=kp_pts1,output=out_path, inspection = inspection)
                    kp2 = kpd.keypoint_detection(teeth_part_n1, res=res, name=f"{name[4:28]}_part_n1", returnIdx = kp_id2, returnPts=kp_pts2,output=out_path, inspection = inspection)
                    q_path = os.path.join(out_path,f'{name.split(".")[0]}_part_n1.json')
                    q1 = run_single1(q_path,kp1,teeth_part_n1,r)
                    q2 = run_single2(q_path,kp2,teeth_part_n1,r)
                q3 = {}
                for k,v in q1.items():
                    q3[k] = list(np.concatenate([Normalize_Data(v),Normalize_Data(q2[k])]))
                with open(q_path, "w") as fp:
                    json.dump(q3 , fp)
                print(f"Keypoint detection: {round((kp_num/len(names))*100,2)}%        7/8", end = "\r")
            
                # Keypoint detection for noisy dentition (0.1)
                mel = 1/round(len(teeth_partial2.points())/teeth_partial2.area(),3)
                teeth_part_n3 = teeth_partial2.clone()
                n3 = 0.3*mel
                teeth_part_n3 = teeth_part_n3.pointGaussNoise(sigma=n3)
                if kp_id1 == kp_id2:
                    kp_id = kp_id1
                    kp_pts = kp_pts1
                    kp = kpd.keypoint_detection(teeth_part_n3, res=res, name=f"{name[4:28]}_part_n3", returnIdx = kp_id, returnPts=kp_pts,output=out_path, inspection = inspection)
                    q_path = os.path.join(out_path,f'{name.split(".")[0]}_part_n3.json')
                    q1 = run_single1(q_path,kp,teeth_part_n3,r)
                    q2 = run_single2(q_path,kp,teeth_part_n3,r)
                else:
                    kp1 = kpd.keypoint_detection(teeth_part_n3, res=res, name=f"{name[4:28]}_part_n3", returnIdx = kp_id1, returnPts=kp_pts1,output=out_path, inspection = inspection)
                    kp2 = kpd.keypoint_detection(teeth_part_n3, res=res, name=f"{name[4:28]}_part_n3", returnIdx = kp_id2, returnPts=kp_pts2,output=out_path, inspection = inspection)
                    q_path = os.path.join(out_path,f'{name.split(".")[0]}_part_n3.json')
                    q1 = run_single1(q_path,kp1,teeth_part_n3,r)
                    q2 = run_single2(q_path,kp2,teeth_part_n3,r)
                q3 = {}
                for k,v in q1.items():
                    q3[k] = list(np.concatenate([Normalize_Data(v),Normalize_Data(q2[k])]))
                with open(q_path, "w") as fp:
                    json.dump(q3 , fp)
                print(f"Keypoint detection: {round((kp_num/len(names))*100,2)}%        8/8", end = "\r")
            
                #keypoint detection for partial 3
                teeth_partial3 = teeth_full.clone()
                theta=0.3
                mat = [[np.cos(theta),-np.sin(theta),0],[np.sin(theta), np.cos(theta), 0],[0,0,1]]
                for i, point in enumerate(teeth_full.points()):
                    teeth_partial3.points()[i] = np.matmul(mat,point)
                teeth_partial3.decimate(fraction=0.80 ,method='quadratic')
                if kp_id1 == kp_id2:
                    kp_id = kp_id1
                    kp_pts = kp_pts1
                    kp = kpd.keypoint_detection(teeth_partial3, res=res, name=f"{name[4:28]}_rot", returnIdx = kp_id, returnPts=kp_pts,output=out_path, inspection = inspection)
                    q_path = os.path.join(out_path,f'{name.split(".")[0]}_rot.json')
                    q1 = run_single1(q_path,kp,teeth_partial3,r)
                    q2 = run_single2(q_path,kp,teeth_partial3,r)
                else:
                    kp1 = kpd.keypoint_detection(teeth_partial3, res=res, name=f"{name[4:28]}_rot", returnIdx = kp_id1, returnPts=kp_pts1,output=out_path, inspection = inspection)
                    kp2 = kpd.keypoint_detection(teeth_partial3, res=res, name=f"{name[4:28]}_rot", returnIdx = kp_id2, returnPts=kp_pts2,output=out_path, inspection = inspection)
                    q_path = os.path.join(out_path,f'{name.split(".")[0]}_rot.json')
                    q1 = run_single1(q_path,kp1,teeth_partial3,r)
                    q2 = run_single2(q_path,kp2,teeth_partial3,r)
                q3 = {}
                for k,v in q1.items():
                    q3[k] = list(np.concatenate([Normalize_Data(v),Normalize_Data(q2[k])]))
                with open(q_path, "w") as fp:
                    json.dump(q3 , fp)
                print(f"Keypoint detection: {round((kp_num/len(names))*100,2)}%        8/8", end = "\r")

## Landmark correspondance

In [ ]:
import json
def convert_to_array(dictionary):
    '''Converts lists of values in a dictionary to numpy arrays'''
    return {k:np.array(v) for k, v in dictionary.items()}
def landmark_correspondence(sfh_dict1, sfh_dict2, method="ratio"):
    """ Function for finding landmark correspondence"""
    """
    Input: two landmark dicts with flat matrices as values
    Output: landmark key correspondence"""
    
    #Initialize
    distance_ratios = []
    warning = False
    
    #Get size order
    if len(sfh_dict1) < len(sfh_dict2):
        shortest = sfh_dict1
        longest = sfh_dict2
    else:
        shortest = sfh_dict2
        longest = sfh_dict1
    
    #Iterate through shortest dict
    for i, (key1, value1) in enumerate(shortest.items()):
        #print("Making landmark correspondence: ",round(((i+1)/len(shortest))*100,2), "%    ", end="\r")
        dist_list = []
        
        #Get distance to all in longest dict
        dist_list = np.linalg.norm((list(longest.values())-value1),axis=1)

        #Sort landmarks according to similarity 
        dists_sorted, idx_matches = zip(*sorted(zip(dist_list,[idx for idx in range(len(dist_list))])))
        
        #Ratio between best match and second best match
        if method == "ratio":
            if dists_sorted[1] != 0 and dists_sorted[1] != dists_sorted[0]:
                L2_ratio = dists_sorted[0]/dists_sorted[1]
                distance_ratios.append(L2_ratio)
            else:
                L2_ratio = 1
                distance_ratios.append(L2_ratio)
        else:
            distance_ratios.append(dists_sorted[0])


    #Score
    score = np.subtract(*np.percentile(distance_ratios, [75, 25]))
    #print("Final Score: ",score)
    
    #Sanity check
    if min(distance_ratios) < 0:
        print("\nWARNING! distance ratio below 0")
        warning = True


    return distance_ratios, score, warning

In [ ]:

for method in ["shot_ecsad_r2","shot_usc_r2","shot_sfh_r2","shot_rops_r2","ecsad_usc_r2","ecsad_sfh_r2","ecsad_rops_r2","usc_sfh_r2","usc_rops_r2","sfh_rops_r2"]: #
    print(f"Working with method {method}")
    
    # Get all combinations of dental input
    in_path = "Path/to/inpath/"
    names = os.listdir(in_path)
    inspection = False
    kp_id = False
    kp_pts = True
    
    for r in [1]: #[1,1.5,2]
        if method == "sfh":
            bins = 10 
            method_name = f"sfh_r{r}_bins{bins}"
        elif method == "ecsad":
            method_name = f"ecsad_r{r}"
        elif method == "rops":
            method_name = f"rops_r{r}"
        elif method == "shot":
            method_name = f"shot_r{r}"
        elif method == "usc":
            method_name = f"usc_r{r}"
        else:
            method_name = method

        in_path = f"/path/to/inpath/{method_name}/"
        names = os.listdir(in_path)
        
        if len(names) == 48:
        
            distances_match =  []
            scores_match = []
            distances_mismatch = []
            scores_mismatch = []
            count = 0

            for q_num, q_name in enumerate(names):
                for t_num, t_name in enumerate(names):
                    count += 1
                    print(f"Keypoint Correspondance: {round((count/(len(names)*len(names)))*100,2)}%        ", end = "\r")
            
                    if t_num != q_num and len(q_name.split("part1")) == 1 and len(t_name.split("part1")) == 1:
                    
                        # Load data
                        q_f = open(os.path.join(in_path,q_name),'rb')
                        q_rep = json.load(q_f)
                        q_rep = convert_to_array(q_rep)
                        t_f = open(os.path.join(in_path,t_name),'rb')
                        t_rep = json.load(t_f)
                        t_rep = convert_to_array(t_rep)
                    
                        #Get difference
                        distances, score, warning = landmark_correspondence(q_rep, t_rep, method="ratio")
                    
                        #Check matches
                        if str(q_name.split("_")[:3]) == str(t_name.split("_")[:3]):
                            distances_match.append(distances)
                            scores_match.append(score)
                        else:
                            distances_mismatch.append(distances)
                            scores_mismatch.append(score)
                        
                        q_f.close()
                        t_f.close()
                        
                        
                        
            # Plot the distance  outcome
            print("\nMatches: ",len(distances_match))
            print("Mismatches: ",len(distances_mismatch))
            for red in distances_mismatch:
                plt.hist(red, bins=100, histtype = 'step', lw=2, color="red", alpha=0.2,density = True)
            for blue in distances_match:
                plt.hist(blue, bins=100, histtype = 'step', lw=2, color="cornflowerblue", alpha=0.2,density = True)
            plt.title("L2 Ratios    Method: " + str(method_name))
            plt.xlabel('Ratio of distances (closest/next closest)')
            plt.ylabel('Histogram density')
            red_patch = mpatches.Patch(color='red', label='Mismatch')
            blue_patch = mpatches.Patch(color='cornflowerblue', label='Match')
            plt.legend(handles=[red_patch, blue_patch],loc='upper left')
            plt.savefig(os.path.join(in_path,f'hist_dist_{method_name}.pdf'))
            plt.clf()
            
            # Plot the distance  outcome
            mark50_match = []
            mark50_mismatch = []
            for red in distances_mismatch:
                mismatch_cum = plt.hist(red, bins=100, histtype = 'step', lw=2, color="red", alpha=0.2,density = True,cumulative=True)
                mark50_mismatch.append(mismatch_cum[1][[ n for n,i in enumerate(mismatch_cum[0]) if i>=0.5 ][0]])
            for blue in distances_match:
                match_cum = plt.hist(blue, bins=100, histtype = 'step', lw=2, color="cornflowerblue", alpha=0.2,density = True,cumulative=True)
                mark50_match.append(match_cum[1][[ n for n,i in enumerate(match_cum[0]) if i>=0.5 ][0]])
            plt.title("L2 Ratios    Method: " + str(method_name))
            plt.xlabel('Ratio of distances (closest/next closest) (cummulative)')
            plt.ylabel('Cumulative histogram density')
            red_patch = mpatches.Patch(color='red', label='Mismatch')
            blue_patch = mpatches.Patch(color='cornflowerblue', label='Match')
            plt.legend(handles=[red_patch, blue_patch],loc='upper left')
            plt.savefig(os.path.join(in_path,f'hist_cum_dist_{method_name}.pdf'))
            plt.clf()
                
            # Plot the scores outcome
            q_bins = round((max(scores_match)-min(scores_match))*50)
            plt.hist(scores_match,bins = q_bins, histtype = 'step', lw=2, color="cornflowerblue", alpha=0.8)
            plt.hist(scores_mismatch,bins = 10, histtype = 'step', lw=2, color="red", alpha=0.8)
            plt.title("IQRs    Method: " + str(method_name))
            plt.xlabel('IQR')
            plt.ylabel('Counts')
            red_patch = mpatches.Patch(color='red', label='Mismatch')
            blue_patch = mpatches.Patch(color='cornflowerblue', label='Match')
            plt.legend(handles=[red_patch, blue_patch],loc='upper right')
            plt.savefig(os.path.join(in_path,f'hist_score_{method_name}.pdf'))
            plt.clf()
            
            # Plot the scores outcome
            q_bins = round((max(mark50_match)-min(mark50_match))*50)
            plt.hist(mark50_match,bins = q_bins, histtype = 'step', lw=2, color="cornflowerblue", alpha=0.8)
            plt.hist(mark50_mismatch,bins = 10, histtype = 'step', lw=2, color="red", alpha=0.8)
            plt.title("Method: " + str(method_name))
            plt.xlabel('50% cumulative mark')
            plt.ylabel('Counts')
            red_patch = mpatches.Patch(color='red', label='Mismatch')
            blue_patch = mpatches.Patch(color='cornflowerblue', label='Match')
            plt.legend(handles=[red_patch, blue_patch],loc='upper left')
            plt.savefig(os.path.join(in_path,f'hist_score_50_mark_{method_name}.pdf'))
            plt.clf()
            
        else:
            print("Not done calculating. Skip")
        
        

